In [1]:
import matplotlib.pyplot as plt
import numpy as np
from graphtime import GroupFusedGraphLasso, IndepFusedGraphLasso
from graphtime.utils import get_edges, get_change_points, plot_data_with_cps
from graphtime.utils import precision, recall, f1_score

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 7.0)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
y = np.load('data/y.npy')
sigma = np.load('data/sigma.npy')
sigma_inv = np.load('data/sigma_inv.npy')
T = 90 # Steps
K = 2 # Changepoints
P = 10 # Variables
M = 5 # Active Edges
eps = 0.000001 # Edge threshold epsilon

In [5]:
edges = get_edges(sigma_inv[0], eps, P)
y_true = get_change_points(sigma_inv, eps, T, P)

### Run Estimation with GFGL

In [6]:
verbose = False
tol = 1e-4
max_iter = 500
gammas = [1, 1, 1] # gamma_V1, gamma_V2, gamma_W
lambda1 = 0.15
lambda2 = 25
gfgl = GroupFusedGraphLasso(lambda1, lambda2, gammas[0], gammas[1], gammas[2], tol, max_iter, verbose)
gfgl.fit(y);

### F, Precision, Recall
According to [this paper](https://projecteuclid.org/download/pdfview_1/euclid.ejs/1351865118), we treat graph
recovery as a classification problem of edges. For the mathematical definition, see paper p.19.

In [7]:
precision(gfgl.sparse_Theta, sigma_inv)

0.11178451178451154

In [8]:
recall(gfgl.sparse_Theta, sigma_inv)

1.0

In [9]:
f1_score(gfgl.sparse_Theta, sigma_inv)

0.20109024833434244

### Exhaustive Grid Search with performance estimation

In [10]:
def performance(G_est, G_true):
    prec = precision(G_est, G_true)
    rec = recall(G_est, G_true)
    f1 = f1_score(G_est, G_true)
    return prec, rec, f1

In [17]:
def grid_search(y, G_true, lam1s, lam2s, tol=1e-4, max_iter=500, gamma1=1, gamma2=1, gammaw=1):
    performances = dict()
    for lam1 in lam1s:
        for lam2 in lam2s:
            gfgl = GroupFusedGraphLasso(lam1, lam2, gamma1, gamma2, gammaw, tol, max_iter)
            gfgl.fit(y)
            performances[(lam1, lam2)] = performance(gfgl.sparse_Theta, G_true)
    return performances

In [18]:
lam1s = [0.1, 0.15, 0.2]
lam2s = [20, 25, 30]
pfs = grid_search(y, sigma_inv, lam1s, lam2s)

In [20]:
pfs

{(0.1, 20): (0.1111111111111109, 1.0, 0.19999999999999965),
 (0.1, 25): (0.11156004489337801, 1.0, 0.20072697899838413),
 (0.1, 30): (0.1111111111111109, 1.0, 0.19999999999999965),
 (0.15, 20): (0.11136363636363614, 1.0, 0.20040899795500985),
 (0.15, 25): (0.11178451178451154, 1.0, 0.20109024833434244),
 (0.15, 30): (0.1111111111111109, 1.0, 0.19999999999999965),
 (0.2, 20): (0.11136363636363614, 1.0, 0.20040899795500985),
 (0.2, 25): (0.11178451178451154, 1.0, 0.20109024833434244),
 (0.2, 30): (0.11189674523007832, 1.0, 0.20127182800040336)}